In [1]:
## imports 

# For testing detectron2 mask r-cnn with COCO 

import detectron2 

#Setup logger
from detectron2.utils.logger import setup_logger
setup_logger()

#Some common libraries 
import numpy as np 
import os,json,cv2,random 


#Detectron2 utilities 

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg 
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


/Users/HeleneSemb/opt/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys 
sys.path.append(r"/Users/HeleneSemb/Documents/chipsogdip/kode")
from kode.training import *
from kode.dataset import * 
from detectron2.structures import BoxMode 
from detectron2.utils.visualizer import ColorMode 
from detectron2.config import get_cfg

#for d in ["train", "val"]:
 #   DatasetCatalog.register("damage_" + d, lambda d=d: load_damage_dicts(r"/Volumes/helensem/Master/Labeled_pictures", d))
  #  MetadataCatalog.get("damage_" + d).set(thing_classes=["damage"])

cfg = get_cfg() 
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("damage_train")
cfg.DATASETS.TEST = ()
cfg.MODEL.DEVICE = "cpu"
cfg.DATALOADER.NUM_WORKERS = 2 
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 1 
cfg.SOLVER.BASE_LR = 0.00025 
cfg.SOLVER.MAX_ITER = 300 
cfg.SOLVER.STEPS = [] 
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 

cfg.OUTPUT_DIR = "/Volumes/helensem/Master/output/run1/resnet50"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7


predictor = DefaultPredictor(cfg) 
damage_metadata = MetadataCatalog.get("damage_train")

dataset_dicts = load_damage_dicts(r"/Volumes/helensem/Master/data", "train") #our testing folder
for d in random.sample(dataset_dicts, 1): 
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                    metadata = damage_metadata,
                    scale = 0.5,
                    instance_mode = ColorMode.IMAGE)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite(r"/Users/HeleneSemb/Documents/images/img.jpg", out.get_image()[:,:,::-1]) #* Save images 
    cv2.imshow("image",out.get_image()[:,:,::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[02/27 11:30:20 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Volumes/helensem/Master/output/run1/resnet50/model_final.pth ...
[02/27 11:30:20 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Volumes/helensem/Master/output/run1/resnet50/model_final.pth ...
/Volumes/helensem/Master/data/train/000004
/Volumes/helensem/Master/data/train/000004/1.png
/Volumes/helensem/Master/data/train/000004/masks/grov_merking_0.png
/Volumes/helensem/Master/data/train/000006
/Volumes/helensem/Master/data/train/000006/1.png
/Volumes/helensem/Master/data/train/000006/masks/grov_merking_0.png
/Volumes/helensem/Master/data/train/000001
/Volumes/helensem/Master/data/train/000001/1.png
/Volumes/helensem/Master/data/train/000001/masks/grov_merking_0.png
/Volumes/helensem/Master/data/train/000001/masks/grov_merking_2.png
/Volumes/helensem/Master/data/train/000001/masks/grov_merking_1.png
/Volumes/helensem/Master/data/train/000001/masks/grov_merking_3.png


In [12]:
import os 
  
# importing shutil module 
import shutil 
  
# path 
path = r"/Volumes/helensem/Master/Labeled_pictures/val/0/0.jpeg"


dest = r"/Volumes/helensem/Master/output/run1/resnet101/images"


#source = os.path.join(path, image)
#print(source)
image_id = next(os.walk((os.path.dirname(path))))[2][0]
print(image_id)
full_output = os.path.join(dest, image_id)
#full_output = output + ".jpg"
print(full_output)


#     
#     destination = os.path.join(dest, image)
#     if image.endswith(".jpg"):
#         shutil.move(source,destination)


0.jpeg
/Volumes/helensem/Master/output/run1/resnet101/images/0.jpeg


In [1]:
import numpy as np 

array = np.zeros((1,2,3))
array = array.T 
print(array.shape)

(3, 2, 1)


In [2]:
import sys 
sys.path.append(r"/Users/HeleneSemb/Documents/chipsogdip/kode")
from kode.training import *
from kode.dataset import * 
import cv2 
from kode.eval import combine_masks_to_one

path = r"/Volumes/helensem/Master/Labeled_pictures/val/0/0.jpeg"
mask_path = os.path.dirname(path)
mask_path = os.path.join(mask_path, "masks")
print(mask_path)
image = cv2.imread(path)
cfg = config()
cfg.MODEL.WEIGHTS = os.path.join("/Volumes/helensem/Master/output/run1/resnet101", "model_final.pth")
cfg.MODEL.DEVICE = "cpu"
predictor = DefaultPredictor(cfg)
output = predictor(image)


mask_gt = load_mask(mask_path)
predicted_masks = output['instances'].to("cpu").pred_masks.numpy()
predicted_masks = predicted_masks.T




print("img: ", image.shape)
print("mask: ", mask_gt.shape)
print("pred: ", predicted_masks.shape)

mask_gt = combine_masks_to_one(mask_gt)
mask_pred = combine_masks_to_one(predicted_masks)

print(mask_gt.shape)
print(mask_pred.shape)

/Users/HeleneSemb/opt/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Volumes/helensem/Master/Labeled_pictures/val/0/masks
[02/28 11:40:48 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Volumes/helensem/Master/output/run1/resnet101/model_final.pth ...
[02/28 11:40:48 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /Volumes/helensem/Master/output/run1/resnet101/model_final.pth ...


/Users/HeleneSemb/opt/miniconda3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


/Volumes/helensem/Master/Labeled_pictures/val/0/masks/grov_merking_6.png
/Volumes/helensem/Master/Labeled_pictures/val/0/masks/grov_merking_7.png
/Volumes/helensem/Master/Labeled_pictures/val/0/masks/grov_merking_4.png
/Volumes/helensem/Master/Labeled_pictures/val/0/masks/grov_merking_2.png
/Volumes/helensem/Master/Labeled_pictures/val/0/masks/grov_merking_1.png
/Volumes/helensem/Master/Labeled_pictures/val/0/masks/grov_merking_9.png
/Volumes/helensem/Master/Labeled_pictures/val/0/masks/grov_merking_8.png
/Volumes/helensem/Master/Labeled_pictures/val/0/masks/grov_merking_5.png
/Volumes/helensem/Master/Labeled_pictures/val/0/masks/grov_merking_10.png
/Volumes/helensem/Master/Labeled_pictures/val/0/masks/grov_merking_3.png
img:  (512, 512, 3)
mask:  (512, 512, 10)
pred:  (512, 512, 54)
(512, 512, 1)
(512, 512, 1)


In [33]:
from detectron2.structures import BoxMode 


def find_contours(sub_mask):
    assert sub_mask is not None, "file could not be read, check with os.path.exists()"
    imgray = cv2.cvtColor(sub_mask, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours[0]


def create_image_annotation(file_name, width, height, image_id):
    return {
        "id": image_id,
        "width": width,
        "height": height,
        "file_name": file_name,
    }


def create_annotation_format(contour):
    return {
        "iscrowd": 0,
        "segmentation": [contour.flatten().tolist()],
        "bbox": cv2.boundingRect(contour),
        "bbox_mode": BoxMode.XYWH_ABS,
        "category_id": 0,
    }

In [36]:
import skimage


def load_damage_dicts(dataset_dir, subset): 
    dataset_dicts = []

    assert subset in ["train", "val"]
    dataset_dir = os.path.join(dataset_dir, subset)
    image_ids = next(os.walk(dataset_dir))[1]
    for image_id in image_ids:

        image_dir = os.path.join(dataset_dir, image_id)
        (_, _, file_names) = next(os.walk(image_dir))
        file_name = file_names[0]
        image_path = os.path.join(image_dir, file_name)

        height, width = cv2.imread(image_path).shape[:2]
        record = create_image_annotation(image_path, width, height, image_id)

        mask_dir = os.path.join(image_dir, 'masks')
        objs = []
        for f in next(os.walk(mask_dir))[2]:
            if f.endswith('.png') and ('corrosion' or 'grov_merking' in f):
                mask_path = os.path.join(mask_dir, f)
                mask = cv2.imread(mask_path)
                contour = find_contours(mask)
                obj = create_annotation_format(contour)
                objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts


for d in ["train", "val"]:
    DatasetCatalog.register("damage_" + d, lambda d=d: load_damage_dicts(r"/Users/HeleneSemb/Documents/Master/chipsogdip", "train"))
    MetadataCatalog.get("damage_" + d).set(thing_classes=["damage"])

balloon_metadata = MetadataCatalog.get("damage_train")

In [ ]:
dataset_dicts = load_damage_dicts(r"/Users/HeleneSemb/Documents/Master/chipsogdip", "train")
for d in random.sample(dataset_dicts, 1): 
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:,:,::-1], metadata = balloon_metadata, scale =0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2.imshow("imageout", out.get_image()[:,:,::-1])
    cv2.waitKey(0)
    # closing all open windows
    cv2.destroyAllWindows()

In [2]:
%load_ext tensorboard
%tensorboard --logdir /Volumes/helensem/Master/output/run1/resnet50


Reusing TensorBoard on port 6006 (pid 6407), started 0:14:21 ago. (Use '!kill 6407' to kill it.)